In [1]:
import numpy as np

# Алгоритм Витерби

In [2]:
def Viterbi(observed, event_codes, p, trans_prob, event_prob):
    ob_seq = [event_codes[o] for o in observed]
    n_events = len(observed)
    n_states = trans_prob.shape[0]
    delta = np.zeros((n_states, n_events))
    
    for i in range(n_states):
        delta[i, 0] = p[i] * event_prob[i, ob_seq[0]]
        
    for t in range(1, n_events):
        for i in range(n_states):
            delta[i, t] = max(delta[:, t - 1] * trans_prob[:, i] * event_prob[i, ob_seq[t]])
    
    hidden_st = np.zeros((n_events, ), dtype=int)
    hidden_st[n_events - 1] = np.argmax(delta[:, n_events - 1])
    
    for t in range(n_events - 2, -1, -1):
        hidden_st[t] = np.argmax(delta[:, t].T * trans_prob[:, hidden_st[t + 1]]
                                 * event_prob[:, ob_seq[t]])
        
    return hidden_st

# Алгорим прямого-обратного хода

In [3]:
def FB(observed, event_codes, p, trans_prob, event_prob):
    ob_seq = [event_codes[o] for o in observed]
    n_events = len(observed)
    n_states = trans_prob.shape[0]
    
    alpha = np.zeros((n_states, n_events))
    for i in range(n_states):
        alpha[i, 0] = p[i] * event_prob[i, ob_seq[0]]
        
    for t in range(1, n_events):
        for i in range(n_states):
            alpha[i, t] = alpha[:, t - 1].T @ trans_prob[:, i] * event_prob[i, ob_seq[t]]
    
    beta = np.zeros((n_states, n_events))
    for i in range(n_states):
        beta[i, n_events - 1] = 1
        
    for t in range(n_events - 2, -1, -1):
        for i in range(n_states):
            beta[i, t] = trans_prob[i, :] @ (event_prob[:, ob_seq[t + 1]] * beta[:, t + 1])
       
    end_prob = sum(alpha[:, n_events- 1])
    P = np.zeros((n_states, n_events))
    for t in range(n_events):
        P[:, t] = alpha[:, t] * beta[:, t] / end_prob
        
    return P

In [4]:
def print_FB_res(P):
    n = P.shape[0]
    m = P.shape[1]
    
    k = int(m / 2)
    print("\nВероятности скрытых состояний, предсказанные алгоритмом прямого-обратного хода:")
    
    for i in range(2):
        left = 0 ** (1 - i) * k ** i
        right = k ** (1 - i) * m ** i
        for j in range(n):
            for l in range(left, right):
                print("{:.2f}".format(P[j, l]), end="|")
            print()
        print()
        
    print("Наиболее вероятные скрытые состояния:")
    for i in range(m):
        print(np.argmax(P[:, i]), end=" ")

In [5]:
def print_Vit_res(st):
    print("Скрытые состояния, предсказанные алгоритмом Витерби:")
    for s in st:
        print(s, end=" ")
    print()

# Тесты 1-2

In [6]:
def test1():
    event_codes = {"О": 0, "Р": 1}
    p = np.asarray([0.5, 0.5])
    trans_prob = np.asarray([[0.8, 0.2],
                             [0.2, 0.8]])
    event_prob = np.asarray([[0.5, 0.5],
                             [0.1, 0.9]])
    print("Последовательность наблюдений:\n{}\n".format(" ".join("ОРОРОРООРРРРРРРРРРОООООООО")))
    print_Vit_res(Viterbi("ОРОРОРООРРРРРРРРРРОООООООО", event_codes, p, trans_prob, event_prob))
    print_FB_res(FB("ОРОРОРООРРРРРРРРРРОООООООО", event_codes, p, trans_prob, event_prob))

In [7]:
test1()

Последовательность наблюдений:
О Р О Р О Р О О Р Р Р Р Р Р Р Р Р Р О О О О О О О О

Скрытые состояния, предсказанные алгоритмом Витерби:
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 

Вероятности скрытых состояний, предсказанные алгоритмом прямого-обратного хода:
0.86|0.77|0.88|0.78|0.89|0.80|0.91|0.86|0.46|0.27|0.19|0.15|0.14|
0.14|0.23|0.12|0.22|0.11|0.20|0.09|0.14|0.54|0.73|0.81|0.85|0.86|

0.14|0.15|0.19|0.28|0.47|0.89|0.96|0.98|0.98|0.98|0.98|0.97|0.94|
0.86|0.85|0.81|0.72|0.53|0.11|0.04|0.02|0.02|0.02|0.02|0.03|0.06|

Наиболее вероятные скрытые состояния:
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 

In [8]:
def test2():
    event_codes = {"О": 0, "Р": 1}
    p = np.asarray([0.5, 0.5])
    trans_prob = np.asarray([[0.5, 0.5],
                             [0.5, 0.5]])
    event_prob = np.asarray([[0.5, 0.5],
                             [0.51, 0.49]])
    print("Последовательность наблюдений:\n{}\n".format(" ".join("ОРОРОРООРРРРРРРРРРОООООООО")))
    print_Vit_res(Viterbi("ОРОРОРООРРРРРРРРРРОООООООО", event_codes, p, trans_prob, event_prob))
    print_FB_res(FB("ОРОРОРООРРРРРРРРРРОООООООО", event_codes, p, trans_prob, event_prob))

In [9]:
test2()

Последовательность наблюдений:
О Р О Р О Р О О Р Р Р Р Р Р Р Р Р Р О О О О О О О О

Скрытые состояния, предсказанные алгоритмом Витерби:
1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 

Вероятности скрытых состояний, предсказанные алгоритмом прямого-обратного хода:
0.50|0.51|0.50|0.51|0.50|0.51|0.50|0.50|0.51|0.51|0.51|0.51|0.51|
0.50|0.49|0.50|0.49|0.50|0.49|0.50|0.50|0.49|0.49|0.49|0.49|0.49|

0.51|0.51|0.51|0.51|0.51|0.50|0.50|0.50|0.50|0.50|0.50|0.50|0.50|
0.49|0.49|0.49|0.49|0.49|0.50|0.50|0.50|0.50|0.50|0.50|0.50|0.50|

Наиболее вероятные скрытые состояния:
1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 